In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import django
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'dashboard.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import pandas as pd
from data.voterdb import VoterDb

In [4]:
db = VoterDb()

In [5]:
results = db.fetchall(f"""
    select vs.voter_id, vs.date_added, vs.registration_date,
     vd.race_id, vd.year_of_birth, vd.gender
    from voter_status as vs
    join voter_demographics vd on vs.voter_id = vd.voter_id
    where vs.status = 'A'
""")

In [6]:
df = pd.DataFrame.from_records(results, columns=['voter_id', 'date_added', 'registration_date',
                                                 'race_id', 'year_of_birth', 'gender'])


In [8]:
df = df.assign(date_added=pd.to_datetime(df.date_added, errors='coerce', format='%Y%m%d'),
               registration_date=pd.to_datetime(df.registration_date, errors='coerce', format='%Y-%m-%d'))

In [9]:
df.head()

,voter_id,date_added,registration_date,race_id,year_of_birth,gender
0,01396328,1987-06-12,1987-06-12,WH,1969,M
1,01396510,1988-09-28,1988-09-28,WH,1947,F
2,01396535,1984-10-08,1984-10-08,WH,1947,M
3,01396571,1979-03-20,1979-03-20,WH,1951,F
4,01396606,1976-03-20,1976-03-20,WH,1948,M


In [10]:
from datetime import datetime

oct_2016 = datetime.strptime('20161001', '%Y%m%d')
oct_2020 = datetime.strptime('20201001', '%Y%m%d')

df_2016 = df[(df.date_added <= oct_2016) |  (df.registration_date <= oct_2016)]
df_2020 = df[(df.date_added <= oct_2020) |  (df.registration_date <= oct_2020)]

In [12]:
mean_age_2016 = (2016 - df_2016.year_of_birth).mean()

In [13]:
mean_age_2020 = (2020 - df_2020.year_of_birth).mean()

In [15]:
df_2020_2016 = df_2020[(df_2020.date_added > oct_2016)]

In [16]:
mean_age_2020_2016 = (2020 - df_2020_2016.year_of_birth).mean()

In [19]:
df_2016_wh = len(df_2016[df_2016.race_id == 'WH'].index)
df_2016_nh = len(df_2016[df_2016.race_id != 'WH'].index)
df_2020_wh = len(df_2020[df_2020.race_id == 'WH'].index)
df_2020_nh = len(df_2020[df_2020.race_id != 'WH'].index)

In [21]:
df_2016_nh/len(df_2016.index)

0.4318144753183718

In [22]:
df_2020_nh/len(df_2020.index)

0.4643261572921852

In [23]:
df_2020_2016_nh = len(df_2020_2016[df_2020_2016.race_id != 'WH'].index)

In [24]:
df_2020_2016_nh/len(df_2020_2016.index)

0.5724153239231988

In [25]:
6428800 - 4942178

1486622

In [26]:
df_2020_2016_f = len(df_2020_2016[df_2020_2016.gender == 'F'].index)

In [27]:
df_2020_2016_f/len(df_2020_2016.index)

0.4907469252267614

In [28]:
df_2016_f = len(df_2016[df_2016.gender == 'F'].index)/len(df_2016.index)

In [29]:
df_2020_f = len(df_2020[df_2020.gender == 'F'].index)/len(df_2020.index)